In [ ]:
import pandas as pd
import numpy as np
import lightgbm as lgb
import gc
import os
from pathlib import Path

In [ ]:
GANANCIA_ACIERTO = 780000
COSTO_ESTIMULO = -20000

In [ ]:
def calcular_ganancia_curva(y_true, y_pred):
    """
    Calcula la curva de ganancia acumulada ordenando por probabilidad descendente.
    
    Args:
        y_true: Array con las clases reales (0 o 1).
        y_pred: Array con las probabilidades predichas (0.0 a 1.0).
        
    Returns:
        ganancia_acumulada: Array numpy donde el índice i representa la ganancia
                            acumulada si cortáramos en el envío i+1.
    """
    # 1. Ordenar los indices por probabilidad predicha (de mayor a menor)
    idx_sorted = np.argsort(y_pred)[::-1]
    y_true_sorted = y_true[idx_sorted]
    
    # 2. Calcular la ganancia individual de cada envío
    #    - Si es BAJA (1): Ganamos el valor del cliente + Costo del estímulo (negativo)
    #    - Si es CONTINUA (0): Solo incurrimos en el Costo del estímulo
    valores_individuales = np.where(
        y_true_sorted == 1,
        GANANCIA_ACIERTO + COSTO_ESTIMULO, # Ej: 78000 + (-2000) = 76000
        COSTO_ESTIMULO                     # Ej: -2000
    )
    
    # 3. Acumular la ganancia (cumsum)
    ganancia_acumulada = np.cumsum(valores_individuales)
    
    return ganancia_acumulada

In [ ]:
# --- Paths Base (Ajustados a tu estructura) ---
BUCKET_PATH = "/home/sanmartinofacundo/buckets/b1"
EXP_ROOT = os.path.join(BUCKET_PATH, "exp")

# --- Configuración del Experimento ---
PARAM = {
    "experimento": "z_us50__python",
    "semilla_primigenia": 974411,
    
    # TU NUEVO INPUT PATH (Parquet)
    "input_path": "/home/sanmartinofacundo/buckets/b1/features/fe_v10/competencia_03_fe_v10.parquet",
    
    "output_path": EXP_ROOT,
    
    # Meses que definiste anteriormente
    "train_meses": [
        201901, 201902, 201903, 201904, 201905, 201906,
        201907, 201908, 201909, 201910, 201911, 201912,
        202001, 202002, 202003, 202004, 202005, 202006,
        202007, 202008, 202009, 202010, 202011, 202012,
        202101, 202102, 202103, 202104, 202105
    ],
    "future_mes": [202107], # Ajustado a tu validación externa o test
    
    "undersampling": 0.50,
    "qcanaritos": 5,
}

# Crear directorios
experimento_folder = os.path.join(PARAM["output_path"], PARAM["experimento"])
os.makedirs(experimento_folder, exist_ok=True)
os.makedirs(os.path.join(experimento_folder, "kaggle"), exist_ok=True)

print(f"Iniciando: {PARAM['experimento']}")
print(f"Input: {PARAM['input_path']}")

In [ ]:
# --- Carga de Datos ---
print("Cargando dataset (formato Parquet)...")

# Lectura directa (pandas detecta pyarrow o fastparquet automáticamente)
df = pd.read_parquet(PARAM["input_path"])

print(f"Dimensiones cargadas: {df.shape}")

In [ ]:
# --- 4. Producción y Undersampling ---

# Filtrar dataset para entrenamiento final
df_train = df[df['foto_mes'].isin(PARAM['train_meses'])].copy()

# Undersampling
np.random.seed(PARAM['semilla_primigenia'])
df_train['azar'] = np.random.uniform(0, 1, size=len(df_train))

# Condición: Mantener todos los BAJA+1 y BAJA+2, y el 50% de CONTINUA
cond_train = (df_train['azar'] <= PARAM['undersampling']) | \
             (df_train['clase_ternaria'].isin(["BAJA+1", "BAJA+2"]))

df_train = df_train[cond_train].copy()
df_train.drop(columns=['azar'], inplace=True)

# Target Engineering (Binaria)
# BAJA+1 y BAJA+2 -> 1, CONTINUA -> 0
df_train['clase01'] = np.where(df_train['clase_ternaria'].isin(["BAJA+1", "BAJA+2"]), 1, 0)

print(f"Registros de entrenamiento final: {len(df_train)}")
print(df_train['clase01'].value_counts())

In [ ]:
# --- 5. zLightGBM: Canaritos ---
# IMPORTANTE: En zLightGBM los canaritos deben ser las PRIMERAS columnas.

print(f"Agregando {PARAM['qcanaritos']} canaritos...")
canaritos_cols = []
for i in range(1, PARAM['qcanaritos'] + 1):
    col_name = f"canarito_{i}"
    # Generamos vector aleatorio
    df_train[col_name] = np.random.uniform(0, 1, size=len(df_train))
    canaritos_cols.append(col_name)

# Reordenar columnas: Canaritos primero + Features originales
features_reales = [c for c in df_train.columns if c not in canaritos_cols + ['clase_ternaria', 'clase01', 'training']]
# Nota: 'training' no existe aquí porque filtramos el df, pero lo excluimos por seguridad
cols_finales_ordenadas = canaritos_cols + features_reales

# Crear Dataset de LightGBM
dtrain = lgb.Dataset(
    data=df_train[cols_finales_ordenadas],
    label=df_train['clase01'],
    free_raw_data=False
)

# Limpiar df_train para liberar RAM antes de entrenar
del df_train
gc.collect()

In [ ]:
# --- 6. Configuración y Entrenamiento ---

# Parámetros zLightGBM
params = {
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'custom', # zLGBM usa esto a veces, o binary_logloss
    'boost_from_average': True,
    'force_row_wise': True,
    'verbosity': -1,
    'seed': PARAM['semilla_primigenia'],
    'max_bin': 31,
    'min_data_in_leaf': 20,
    'num_leaves': 999,
    'learning_rate': 1.0,      # High learning rate + gradient_bound
    'feature_fraction': 0.5,
    
    # Hiperparámetros específicos de zLightGBM
    'gradient_bound': 0.1,     # El freno de mano del score
    'canaritos_count': PARAM['qcanaritos'], # Algunos wrappers de Python requieren pasar este param si no lo detecta solo
}

print("Entrenando zLightGBM...")
# Nota: num_boost_round alto porque zLightGBM debería detenerse solo si no puede splitear
model = lgb.train(
    params,
    dtrain,
    num_boost_round=9999
)

# Guardar modelo
model.save_model(os.path.join(experimento_folder, "zmodelo.txt"))

In [ ]:
# --- 7. Scoring y Evaluación de Ganancia ---
print("Evaluando ganancias...")

# Definición de valores de negocio (Ajustar si son distintos)
GANANCIA_ACIERTO = 78000
COSTO_ESTIMULO = -2000

# 1. Preparar datos futuros (Validación)
df_future = df[df['foto_mes'].isin(PARAM['future_mes'])].copy()

# Agregar canaritos (aleatorios)
for col in canaritos_cols:
    df_future[col] = np.random.uniform(0, 1, size=len(df_future))

# 2. Predecir
# Usamos cols_finales_ordenadas (o cols_ordenadas según como te quedó en el paso 5)
prediccion = model.predict(df_future[cols_finales_ordenadas])

# 3. Preparar la Realidad (y_true)
# Solo BAJA+2 genera ganancia. El resto (incluido BAJA+1) es costo si se envía.
y_true = np.where(df_future['clase_ternaria'] == "BAJA+2", 1, 0)

# 4. Calcular Curva de Ganancia Vectorizada
# a. Ordenar predicciones de mayor a menor
idx_sorted = np.argsort(prediccion)[::-1]
y_true_sorted = y_true[idx_sorted]

# b. Calcular ganancia individual de cada envío (vectorizado)
#    Si es BAJA+2 (1) -> Gana 78000 y Paga 2000 = +76000
#    Si es OTRO (0)   -> Solo Paga 2000          = -2000
valores_individuales = np.where(
    y_true_sorted == 1,
    GANANCIA_ACIERTO + COSTO_ESTIMULO,
    COSTO_ESTIMULO
)

# c. Ganancia acumulada
ganancia_acumulada = np.cumsum(valores_individuales)

# 5. Imprimir resultados para cada corte solicitado
CORTES_ENVIO = [9000, 9500, 10000, 10500, 11000, 11500, 12000, 12500, 13000, 13500, 14000, 14500, 15000]

print("\nResultados de Ganancia por Corte:")
print(f"{'Envios':>10} | {'Ganancia Total':>20}")
print("-" * 35)

for corte in CORTES_ENVIO:
    if corte <= len(ganancia_acumulada):
        # El índice es corte - 1 porque el array empieza en 0
        ganancia_corte = ganancia_acumulada[corte - 1]
        print(f"{corte:10d} | {ganancia_corte:20,.0f}")
    else:
        print(f"{corte:10d} | {'No hay suficientes datos':>20}")

# Opcional: Mostrar el mejor corte absoluto encontrado
mejor_indx = np.argmax(ganancia_acumulada)
mejor_ganancia = ganancia_acumulada[mejor_indx]
print("-" * 35)
print(f"MEJOR CORTE: {mejor_indx + 1} envíos | Ganancia: {mejor_ganancia:,.0f}")
print("Finalizado.")

In [ ]:
# # Guardar archivo para Kaggle
# archivo_salida = os.path.join(experimento_folder, "kaggle", f"KA{PARAM['experimento']}_{PARAM['corte_envios']}.csv")
# tb_prediccion[['numero_de_cliente', 'Predicted']].to_csv(archivo_salida, index=False)

# print(f"Archivo generado exitosamente: {archivo_salida}")
# print("¡Éxito en la competencia!")